# Steps to perform SAST using Sonarqube

## Check if Sonarqube is up

In [ ]:
!docker ps | grep sonarqube

## Start the container if not available

In [ ]:
!docker run -d --name sonarqube -p 9000:9000 sonarqube

## Setting variables

In [3]:
%env SONARQUBE_URL=http://localhost:9000
%env USERNAME=admin
%env PASSWORD=sonar
%env YOUR_PROJECT_KEY=struts
%env YOUR_PROJECT_NAME=struts

env: SONARQUBE_URL=http://localhost:9000
env: USERNAME=admin
env: PASSWORD=sonar
env: YOUR_PROJECT_KEY=struts
env: YOUR_PROJECT_NAME=struts


In [4]:
!echo $SONARQUBE_URL

http://localhost:9000


## Creating user token

This token wil be used across multiple projects

In [ ]:
# API_ENDPOINT="$SONARQUBE_URL/api/user_tokens/generate"
!curl -u $USERNAME:$PASSWORD -X POST $SONARQUBE_URL/api/user_tokens/generate -d name=automation_token > /tmp/sonar.json
!jq -r '.token' /tmp/sonar.json > /tmp/token.txt
TOKEN = open('/tmp/token.txt', 'r').read()
%env TOKEN=$TOKEN
!rm /tmp/token.txt /tmp/sonar.json

## Create project

The command below will create a new project using the main branch. If you require a different branch, please visit the sonarqube server page and create the project manually.

In [ ]:
!curl -u "$USERNAME":"$PASSWORD" -X POST "$SONARQUBE_URL/api/projects/create?name=$YOUR_PROJECT_NAME&project=$YOUR_PROJECT_KEY

## Run Scan
Move to your project directory to perform the scan

### For maven builds

In [ ]:
%cd "$BESMAN_ARTIFACT_DIR"
!mvn clean verify sonar:sonar -Dsonar.projectKey="$YOUR_PROJECT_KEY" -Dsonar.projectName="$YOUR_PROJECT_NAME" -Dsonar.host.url="$SONARQUBE_URL" -Dsonar.login="$TOKEN"
%cd -

### For gradle builds

Running an analysis with Gradle is straighforward. You just need to declare the `org.sonarqube` plugin in your `build.gradle` or `build.gradle.kts` file:

**build.gradle**
```json
plugins {
  id "org.sonarqube" version "5.0.0.4638"
}
```

**build.gradle.kts**

```json
plugins {
  id("org.sonarqube") version "5.0.0.4638"
}
```


In [ ]:
%cd "$BESMAN_ARTIFACT_DIR"
!ls
!./gradlew sonar -Dsonar.projectKey="$YOUR_PROJECT_KEY" -Dsonar.projectName='$YOUR_PROJECT_NAME' -Dsonar.host.url="$SONARQUBE_URL" -Dsonar.token="$TOKEN"

### For others(JS, TS, Go, Python, PHP)

In [ ]:
!sonar-scanner -Dsonar.projectKey="$YOUR_PROJECT_KEY" -Dsonar.sources=. -Dsonar.host.url="$SONARQUBE_URL" -Dsonar.token="$TOKEN"

## View Report

Once the assessment is done, you can visit the Sonarqube server and see your project scan report

## Download Report

In [5]:
!curl -u "$USERNAME":"$PASSWORD" -X GET $SONARQUBE_URL/api/issues/search?componentKeys=$YOUR_PROJECT_NAME >> /tmp/report.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  168k    0  168k    0     0   149k      0 --:--:--  0:00:01 --:--:--  149k


In [9]:
!curl -L $SONARQUBE_URL/api/issues/search?componentKeys=$YOUR_PROJECT_NAME&types=VULNERABILITY >> /tmp/report2.json

In [18]:
import requests
import os
import json
# Specify the URL from which you want to fetch JSON data
url = "http://localhost:9000/api/issues/search?componentKeys=struts&types=VULNERABILITY"

name = os.environ.get('USERNAME')
pwd = os.environ.get('PASSWORD')

# Specify your username and password
username = name
password = pwd

# Send a GET request to the URL with username and password
response = requests.get(url, auth=(username, password))

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON response
    json_data = response.json()
    # Now you can work with the JSON data as a Python dictionary
    print(json_data)
else:
    print("Failed to fetch data:", response.status_code)
with open("/tmp/vuln.json", "w") as vuln_report:
    json.dump(json_data,vuln_report, indent=4)


{'total': 6, 'p': 1, 'ps': 100, 'paging': {'pageIndex': 1, 'pageSize': 100, 'total': 6}, 'effortTotal': 105, 'issues': [{'key': '86a3d239-eb81-48e1-88b5-3c8bcfaa7f23', 'rule': 'java:S1989', 'severity': 'MINOR', 'component': 'struts:plugins/portlet/src/main/java/org/apache/struts2/portlet/dispatcher/DispatcherServlet.java', 'project': 'struts', 'line': 40, 'hash': '18d9123759b0b461c73306331d2f1c14', 'textRange': {'startLine': 40, 'endLine': 40, 'startOffset': 44, 'endOffset': 51}, 'flows': [], 'status': 'OPEN', 'message': 'Handle the following exceptions that could be thrown by "include": ServletException, IOException.', 'effort': '20min', 'debt': '20min', 'author': 'jogep@apache.org', 'tags': ['cwe', 'error-handling', 'cert'], 'creationDate': '2011-06-26T08:38:59+0000', 'updateDate': '2024-05-13T12:03:53+0000', 'type': 'VULNERABILITY', 'scope': 'MAIN', 'quickFixAvailable': False, 'messageFormattings': [], 'codeVariants': [], 'cleanCodeAttribute': 'CONVENTIONAL', 'cleanCodeAttributeCate

In [19]:
!cat /tmp/vuln.json

{
    "total": 6,
    "p": 1,
    "ps": 100,
    "paging": {
        "pageIndex": 1,
        "pageSize": 100,
        "total": 6
    },
    "effortTotal": 105,
    "issues": [
        {
            "key": "86a3d239-eb81-48e1-88b5-3c8bcfaa7f23",
            "rule": "java:S1989",
            "severity": "MINOR",
            "component": "struts:plugins/portlet/src/main/java/org/apache/struts2/portlet/dispatcher/DispatcherServlet.java",
            "project": "struts",
            "line": 40,
            "hash": "18d9123759b0b461c73306331d2f1c14",
            "textRange": {
                "startLine": 40,
                "endLine": 40,
                "startOffset": 44,
                "endOffset": 51
            },
            "flows": [],
            "status": "OPEN",
            "message": "Handle the following exceptions that could be thrown by \"include\": ServletException, IOException.",
            "effort": "20min",
            "debt": "20min",
            "author": "jogep@apache

In [20]:
import json

# Load the first JSON file
with open("/tmp/vuln.json", "r") as f1:
    data1 = json.load(f1)

# Load the second JSON file
with open("/tmp/report.json", "r") as f2:
    data2 = json.load(f2)

# Extract the contents of the "issues" list from each file
issues1 = data1.get("issues", [])
issues2 = data2.get("issues", [])

# Append the contents of the "issues" list from the second file to the first file
issues1.extend(issues2)

# Update the "issues" list in the first file with the combined contents
data1["issues"] = issues1

# Write the combined data to a new JSON file
with open("combined_file.json", "w") as f_combined:
    json.dump(data1, f_combined, indent=4)


In [21]:
!cat combined_file.json

{
    "total": 6,
    "p": 1,
    "ps": 100,
    "paging": {
        "pageIndex": 1,
        "pageSize": 100,
        "total": 6
    },
    "effortTotal": 105,
    "issues": [
        {
            "key": "86a3d239-eb81-48e1-88b5-3c8bcfaa7f23",
            "rule": "java:S1989",
            "severity": "MINOR",
            "component": "struts:plugins/portlet/src/main/java/org/apache/struts2/portlet/dispatcher/DispatcherServlet.java",
            "project": "struts",
            "line": 40,
            "hash": "18d9123759b0b461c73306331d2f1c14",
            "textRange": {
                "startLine": 40,
                "endLine": 40,
                "startOffset": 44,
                "endOffset": 51
            },
            "flows": [],
            "status": "OPEN",
            "message": "Handle the following exceptions that could be thrown by \"include\": ServletException, IOException.",
            "effort": "20min",
            "debt": "20min",
            "author": "jogep@apache

In [6]:
!cat /tmp/report.json

{"total":7138,"p":1,"ps":100,"paging":{"pageIndex":1,"pageSize":100,"total":7138},"effortTotal":76812,"issues":[{"key":"538fe7e4-d29b-4763-a177-a64bdf6b439c","rule":"java:S2699","severity":"BLOCKER","component":"struts:core/src/test/java/org/apache/struts2/interceptor/CspInterceptorTest.java","project":"struts","line":124,"hash":"27f9a3c576875d7630531caca4bb4043","textRange":{"startLine":124,"endLine":124,"startOffset":16,"endOffset":34},"flows":[],"status":"OPEN","message":"Add at least one assertion to this test case.","effort":"10min","debt":"10min","author":"lukaszlenart@apache.org","tags":["junit","tests"],"creationDate":"2022-09-01T06:15:48+0000","updateDate":"2024-05-13T12:03:53+0000","type":"CODE_SMELL","scope":"TEST","quickFixAvailable":false,"messageFormattings":[],"codeVariants":[],"cleanCodeAttribute":"TESTED","cleanCodeAttributeCategory":"ADAPTABLE","impacts":[{"softwareQuality":"MAINTAINABILITY","severity":"HIGH"}],"issueStatus":"OPEN"},{"key":"8edd5bf7-3e8c-4875-918d-3dc

In [10]:
!cat /tmp/report2.json